# CCF Report

In [ ]:
import pandas as pd
from IPython.display import display, Markdown, Latex
from typing import List

import src.ccf as ccf

pd.options.display.float_format = '{:,.3f}'.format
pd.options.display.max_rows = None
pd.options.display.max_columns = None

path_to_cache_file = '../cloud-carbon-footprint/packages/cli/estimates.cache.day.json'

In [ ]:
estimates = ccf.load_estimates(path_to_cache_file)

tags: List[str] = []

for column in estimates.columns.tolist():
    if column.startswith('tags.'):
        tags.append(column)

for tag in tags:
    estimates = ccf.normalize_column(estimates, tag)

## Table of contents

In [ ]:
toc = """
1. [Overview](#ccf-report)
2. [Per service](#Per-service)
3. [Per tag](#Per-tag)\n
"""

for i, tag in enumerate(tags):
    toc += f"\t3.{i} [{tag}](#Per-{tag})\n\n"

toc += "4. [Detailed](#Detailed)"

display(Markdown(toc))

Data in this report ranges from:

In [ ]:
first = estimates.iloc[0]['timestamp']
last = estimates.iloc[-1]['timestamp']
print(f"{first} to {last}")

The total power consumption in this time range is:

In [ ]:
summed = ccf.sum_columns(estimates)
kwh = summed[0]['kilowattHours']
display(Markdown(f"{round(kwh, 2)} kWh"))

Power consumption as a time series:

In [ ]:
def plot_time_series(estimates: pd.DataFrame, title: str):
    time_series = ccf.sum_per_column(estimates, 'timestamp')
    time_series = time_series.sort_values(by=['timestamp'])
    time_series = time_series.reset_index()
    time_series[['timestamp', 'kilowattHours']].plot(x='timestamp', ylim=[0, time_series['kilowattHours'].max() * 1.1], ylabel='kWh', title=title)

In [ ]:
plot_time_series(estimates, 'Total power consumption')

## Per service

In [ ]:
per_service = ccf.sum_per_column(estimates, 'serviceName')[['serviceName', 'kilowattHours', 'co2e', 'cost']]
per_service

## Per tag

In [ ]:
for tag in tags:
    per_tag = ccf.sum_per_column(estimates, tag)
    display(Markdown(f'### Per {tag}'))
    display(per_tag[[tag, 'kilowattHours', 'co2e', 'cost']])

## Detailed

In [ ]:
detail_toc = ""

for i, tag in enumerate(tags):
    detail_toc += f"4.{i} [{tag}](#Details-for-{tag})\n\n"

    tag_values = ccf.sum_per_column(estimates, tag)[tag].tolist()
    for j, tag_value in enumerate(tag_values):
        detail_toc += f"4.{i}.{j} [{tag}: {tag_value}](#Details-{tag}:-{tag_value})\n\n"

display(Markdown(detail_toc))

In [ ]:
for tag in tags:
    display(Markdown(f"### Details for {tag}"))

    tag_values = ccf.sum_per_column(estimates, tag)[tag].tolist()
    for tag_value in tag_values:
        display(Markdown(f"#### Details {tag}: {tag_value}"))
        filtered = ccf.filter_estimates(estimates, tag, [tag_value])
        total = ccf.sum_columns(filtered)[0]['kilowattHours']
        display(Markdown(f"Total: {round(total, 2)} kWh"))
        #plot_time_series(filtered, f"Power consumption of {tag}: {tag_value}")

        per_service = ccf.sum_per_column(filtered, 'serviceName')[['serviceName', 'kilowattHours', 'co2e', 'cost']]
        display(per_service)

        for other_tag in tags:
            if tag == other_tag:
                continue
            per_tag = ccf.sum_per_column(filtered, other_tag)
            display(per_tag[[other_tag, 'kilowattHours', 'co2e', 'cost']])